### Creating the SP500 historical database
The goal of this notebook is to construct a SQL database with SP500 historical data from the past 5 years.<br>
We are going to use the IEX API to collect the data from the SP500 listed companies, make some data processing and save it to RDS SQL database on AWS.<br>
This approach was inspired on Kaggle S&P 500 stock data (https://www.kaggle.com/camnugent/sandp500) and part of the code was extracted from the script _getSandP.py_ found on the dataset folder of the challenge.

In [2]:
# import libraries
from datetime import datetime
from concurrent import futures

import pandas as pd
from pandas import DataFrame
import pandas_datareader.data as web
import os

To use IEX API (https://iexcloud.io/docs/api/) trough pandas_datareader, we need to sign up to the service and provide an API key. 


In [3]:
# Define my API key for IEX
os.environ["IEX_API_KEY"] = 'pk_c876f405f2e441cba4bd913e6d066a7f'

We are going to use the list of SP500 from DataHub (https://datahub.io/core/s-and-p-500-companies).
The file 

In [5]:
df_sp500 = pd.read_csv('constituents_csv.csv')
df_sp500.head()

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ACN,Accenture plc,Information Technology


In [8]:
list_sp500 = df_sp500.Symbol.to_list()
len(list_sp500)

505

There are a total of 505 companies listed on Standard & Poor's 500.<br>

Next, we are going to iterate through each company, get their stock data from the past 5 years and concatenate on a dataframe

In [10]:
end_time = datetime.now()
start_time = datetime(end_time.year - 5, end_time.month , end_time.day)
print(end_time)
print(start_time)

2019-10-31 23:38:40.030533
2014-10-31 00:00:00


In [11]:
stock_df = web.DataReader(['MSFT', 'AMZN'] ,'iex', start_time, end_time)

In [12]:
stock_df

Attributes    open             high              low            close  \
Symbols       MSFT     AMZN    MSFT     AMZN    MSFT     AMZN    MSFT   
date                                                                    
2014-10-31   46.94   305.15   46.97   305.72   46.48   301.50   46.95   
2014-11-03   46.89   306.24   47.46   308.01   46.73   303.24   47.44   
2014-11-04   47.30   303.99   47.73   304.25   47.25   300.17   47.57   
2014-11-05   47.80   303.01   47.90   304.00   47.26   294.16   47.86   
2014-11-06   47.86   296.69   48.86   297.00   47.79   292.29   48.70   
...            ...      ...     ...      ...     ...      ...     ...   
2019-10-24  139.39  1771.09  140.42  1788.34  138.67  1760.27  139.94   
2019-10-25  139.34  1697.55  141.14  1764.21  139.20  1695.00  140.73   
2019-10-28  144.40  1748.06  145.67  1778.70  143.51  1742.50  144.19   
2019-10-29  144.08  1774.81  144.50  1777.00  142.65  1755.81  142.83   
2019-10-30  143.52  1760.24  145.00  1782.38  142.79  1759.12  144.61   

Attributes             volume           
Symbols        AMZN      MSFT     AMZN  
date                                    
2014-10-31   305.46  35849656  4551795  
2014-11-03   305.72  23130397  3244150  
2014-11-04   302.81  21530813  3427384  
2014-11-05   296.52  22449594  4674492  
2014-11-06   296.64  33037841  3758121  
...             ...       ...      ...  
2019-10-24  1780.78  37278399  5204350  
2019-10-25  1761.33  25959724  9626402  
2019-10-28  1777.08  35280137  3708851  
2019-10-29  1762.71  20589469  2276855  
2019-10-30  1779.99  18496591  2449405  

[1258 rows x 10 columns]

In [21]:
stock_df = web.DataReader('AMZN','iex', start_time, end_time)

In [13]:
stock_df.T

date                 2014-10-31   2014-11-03   2014-11-04   2014-11-05  \
Attributes Symbols                                                       
open       MSFT           46.94        46.89        47.30        47.80   
           AMZN          305.15       306.24       303.99       303.01   
high       MSFT           46.97        47.46        47.73        47.90   
           AMZN          305.72       308.01       304.25       304.00   
low        MSFT           46.48        46.73        47.25        47.26   
           AMZN          301.50       303.24       300.17       294.16   
close      MSFT           46.95        47.44        47.57        47.86   
           AMZN          305.46       305.72       302.81       296.52   
volume     MSFT     35849656.00  23130397.00  21530813.00  22449594.00   
           AMZN      4551795.00   3244150.00   3427384.00   4674492.00   

date                 2014-11-06   2014-11-07   2014-11-10   2014-11-11  \
Attributes Symbols                                                       
open       MSFT           47.86        48.92        48.65        48.85   
           AMZN          296.69       297.82       300.00       305.94   
high       MSFT           48.86        48.92        49.15        48.93   
           AMZN          297.00       299.99       305.80       312.38   
low        MSFT           47.79        48.29        48.55        48.65   
           AMZN          292.29       296.29       300.00       303.65   
close      MSFT           48.70        48.68        48.89        48.87   
           AMZN          296.64       299.86       305.11       312.01   
volume     MSFT     33037841.00  28000598.00  36370067.00  23445239.00   
           AMZN      3758121.00   2647352.00   2809056.00   3807333.00   

date                 2014-11-12   2014-11-13  ...   2019-10-17   2019-10-18  \
Attributes Symbols                            ...                             
open       MSFT           48.56        48.81  ...       140.95       139.76   
           AMZN          310.75       312.00  ...      1796.49      1787.80   
high       MSFT           48.92        49.65  ...       141.42       140.00   
           AMZN          312.58       317.89  ...      1798.85      1793.98   
low        MSFT           48.52        48.71  ...       139.02       136.56   
           AMZN          308.79       311.51  ...      1782.02      1749.20   
close      MSFT           48.78        49.61  ...       139.69       137.41   
           AMZN          311.51       316.48  ...      1787.48      1757.51   
volume     MSFT     22722123.00  26210433.00  ...  22102797.00  32284428.00   
           AMZN      2182982.00   3686428.00  ...   2713773.00   3366091.00   

date                 2019-10-21   2019-10-22   2019-10-23   2019-10-24  \
Attributes Symbols                                                       
open       MSFT          138.45       138.97       136.88       139.39   
           AMZN         1769.66      1788.15      1761.30      1771.09   
high       MSFT          138.50       140.01       137.45       140.42   
           AMZN         1785.88      1789.78      1770.05      1788.34   
low        MSFT          137.01       136.26       135.61       138.67   
           AMZN         1765.00      1762.00      1742.00      1760.27   
close      MSFT          138.43       136.37       137.24       139.94   
           AMZN         1785.66      1765.73      1762.17      1780.78   
volume     MSFT     20716039.00  27935270.00  31380309.00  37278399.00   
           AMZN      2224902.00   2234425.00   2190380.00   5204350.00   

date                 2019-10-25   2019-10-28   2019-10-29   2019-10-30  
Attributes Symbols                                                      
open       MSFT          139.34       144.40       144.08       143.52  
           AMZN         1697.55      1748.06      1774.81      1760.24  
high       MSFT          141.14       145.67       144.50       145.00  
           AMZN    

In [14]:
stock_df.T.loc[['open']]

date                2014-10-31  2014-11-03  2014-11-04  2014-11-05  \
Attributes Symbols                                                   
open       MSFT          46.94       46.89       47.30       47.80   
           AMZN         305.15      306.24      303.99      303.01   

date                2014-11-06  2014-11-07  2014-11-10  2014-11-11  \
Attributes Symbols                                                   
open       MSFT          47.86       48.92       48.65       48.85   
           AMZN         296.69      297.82      300.00      305.94   

date                2014-11-12  2014-11-13  ...  2019-10-17  2019-10-18  \
Attributes Symbols                          ...                           
open       MSFT          48.56       48.81  ...      140.95      139.76   
           AMZN         310.75      312.00  ...     1796.49     1787.80   

date                2019-10-21  2019-10-22  2019-10-23  2019-10-24  \
Attributes Symbols                                                   
open       MSFT         138.45      138.97      136.88      139.39   
           AMZN        1769.66     1788.15     1761.30     1771.09   

date                2019-10-25  2019-10-28  2019-10-29  2019-10-30  
Attributes Symbols                                                  
open       MSFT         139.34      144.40      144.08      143.52  
           AMZN        1697.55     1748.06     1774.81     1760.24  

[2 rows x 1258 columns]

In [15]:
stock_df.T.loc[['open','high']]

date                2014-10-31  2014-11-03  2014-11-04  2014-11-05  \
Attributes Symbols                                                   
open       MSFT          46.94       46.89       47.30       47.80   
           AMZN         305.15      306.24      303.99      303.01   
high       MSFT          46.97       47.46       47.73       47.90   
           AMZN         305.72      308.01      304.25      304.00   

date                2014-11-06  2014-11-07  2014-11-10  2014-11-11  \
Attributes Symbols                                                   
open       MSFT          47.86       48.92       48.65       48.85   
           AMZN         296.69      297.82      300.00      305.94   
high       MSFT          48.86       48.92       49.15       48.93   
           AMZN         297.00      299.99      305.80      312.38   

date                2014-11-12  2014-11-13  ...  2019-10-17  2019-10-18  \
Attributes Symbols                          ...                           
open       MSFT          48.56       48.81  ...      140.95      139.76   
           AMZN         310.75      312.00  ...     1796.49     1787.80   
high       MSFT          48.92       49.65  ...      141.42      140.00   
           AMZN         312.58      317.89  ...     1798.85     1793.98   

date                2019-10-21  2019-10-22  2019-10-23  2019-10-24  \
Attributes Symbols                                                   
open       MSFT         138.45      138.97      136.88      139.39   
           AMZN        1769.66     1788.15     1761.30     1771.09   
high       MSFT         138.50      140.01      137.45      140.42   
           AMZN        1785.88     1789.78     1770.05     1788.34   

date                2019-10-25  2019-10-28  2019-10-29  2019-10-30  
Attributes Symbols                                                  
open       MSFT         139.34      144.40      144.08      143.52  
           AMZN        1697.55     1748.06     1774.81     1760.24  
high       MSFT         141.14      145.67      144.50      145.00  
           AMZN        1764.21     1778.70     1777.00     1782.38  

[4 rows x 1258 columns]

In [41]:
cols = ['open','high', 'low', 'close', 'volume']
stock_df = web.DataReader('AMZN','iex', start_time, end_time)
concaTdf = pd.concat([stock_df.T.loc['open'],
                      stock_df.T.loc['high'],
                      stock_df.T.loc['low'],
                      stock_df.T.loc['close'],
                      stock_df.T.loc['volume']], axis=1)
#open_series

In [42]:
concaTdf

,open,high,low,close,volume
date,,,,,
2014-10-31,305.15,305.72,301.50,305.46,4551795.0
2014-11-03,306.24,308.01,303.24,305.72,3244150.0
2014-11-04,303.99,304.25,300.17,302.81,3427384.0
2014-11-05,303.01,304.00,294.16,296.52,4674492.0
2014-11-06,296.69,297.00,292.29,296.64,3758121.0
...,...,...,...,...,...
2019-10-24,1771.09,1788.34,1760.27,1780.78,5204350.0
2019-10-25,1697.55,1764.21,1695.00,1761.33,9626402.0
2019-10-28,1748.06,1778.70,1742.50,1777.08,3708851.0


In [18]:
open_series = stock_df.T.loc['open'].loc['AMZN']
open_series

date
2014-10-31     305.15
2014-11-03     306.24
2014-11-04     303.99
2014-11-05     303.01
2014-11-06     296.69
               ...   
2019-10-24    1771.09
2019-10-25    1697.55
2019-10-28    1748.06
2019-10-29    1774.81
2019-10-30    1760.24
Name: AMZN, Length: 1258, dtype: float64

In [19]:
high_series = stock_df.T.loc['high'].loc['AMZN']
high_series

date
2014-10-31     305.72
2014-11-03     308.01
2014-11-04     304.25
2014-11-05     304.00
2014-11-06     297.00
               ...   
2019-10-24    1788.34
2019-10-25    1764.21
2019-10-28    1778.70
2019-10-29    1777.00
2019-10-30    1782.38
Name: AMZN, Length: 1258, dtype: float64

In [26]:
df_concat = pd.concat([open_series, high_series], axis=1)
df_concat.columns = ['open', 'high']
df_concat

,open,high
date,,
2014-10-31,305.15,305.72
2014-11-03,306.24,308.01
2014-11-04,303.99,304.25
2014-11-05,303.01,304.00
2014-11-06,296.69,297.00
...,...,...
2019-10-24,1771.09,1788.34
2019-10-25,1697.55,1764.21
2019-10-28,1748.06,1778.70


In [30]:
df_concat.open

date
2014-10-31     305.15
2014-11-03     306.24
2014-11-04     303.99
2014-11-05     303.01
2014-11-06     296.69
               ...   
2019-10-24    1771.09
2019-10-25    1697.55
2019-10-28    1748.06
2019-10-29    1774.81
2019-10-30    1760.24
Name: open, Length: 1258, dtype: float64

In [13]:
stock_df['open']['MSFT']

date
2014-10-31     46.94
2014-11-03     46.89
2014-11-04     47.30
2014-11-05     47.80
2014-11-06     47.86
               ...  
2019-10-24    139.39
2019-10-25    139.34
2019-10-28    144.40
2019-10-29    144.08
2019-10-30    143.52
Name: MSFT, Length: 1258, dtype: float64

In [7]:
stock_df.columns

MultiIndex([(  'open', 'MSFT'),
            (  'open', 'AMZN'),
            (  'high', 'MSFT'),
            (  'high', 'AMZN'),
            (   'low', 'MSFT'),
            (   'low', 'AMZN'),
            ( 'close', 'MSFT'),
            ( 'close', 'AMZN'),
            ('volume', 'MSFT'),
            ('volume', 'AMZN')],
           names=['Attributes', 'Symbols'])

In [8]:
stock_df.T

date                 2014-10-31   2014-11-03   2014-11-04   2014-11-05  \
Attributes Symbols                                                       
open       MSFT           46.94        46.89        47.30        47.80   
           AMZN          305.15       306.24       303.99       303.01   
high       MSFT           46.97        47.46        47.73        47.90   
           AMZN          305.72       308.01       304.25       304.00   
low        MSFT           46.48        46.73        47.25        47.26   
           AMZN          301.50       303.24       300.17       294.16   
close      MSFT           46.95        47.44        47.57        47.86   
           AMZN          305.46       305.72       302.81       296.52   
volume     MSFT     35849656.00  23130397.00  21530813.00  22449594.00   
           AMZN      4551795.00   3244150.00   3427384.00   4674492.00   

date                 2014-11-06   2014-11-07   2014-11-10   2014-11-11  \
Attributes Symbols                                                       
open       MSFT           47.86        48.92        48.65        48.85   
           AMZN          296.69       297.82       300.00       305.94   
high       MSFT           48.86        48.92        49.15        48.93   
           AMZN          297.00       299.99       305.80       312.38   
low        MSFT           47.79        48.29        48.55        48.65   
           AMZN          292.29       296.29       300.00       303.65   
close      MSFT           48.70        48.68        48.89        48.87   
           AMZN          296.64       299.86       305.11       312.01   
volume     MSFT     33037841.00  28000598.00  36370067.00  23445239.00   
           AMZN      3758121.00   2647352.00   2809056.00   3807333.00   

date                 2014-11-12   2014-11-13  ...   2019-10-17   2019-10-18  \
Attributes Symbols                            ...                             
open       MSFT           48.56        48.81  ...       140.95       139.76   
           AMZN          310.75       312.00  ...      1796.49      1787.80   
high       MSFT           48.92        49.65  ...       141.42       140.00   
           AMZN          312.58       317.89  ...      1798.85      1793.98   
low        MSFT           48.52        48.71  ...       139.02       136.56   
           AMZN          308.79       311.51  ...      1782.02      1749.20   
close      MSFT           48.78        49.61  ...       139.69       137.41   
           AMZN          311.51       316.48  ...      1787.48      1757.51   
volume     MSFT     22722123.00  26210433.00  ...  22102797.00  32284428.00   
           AMZN      2182982.00   3686428.00  ...   2713773.00   3366091.00   

date                 2019-10-21   2019-10-22   2019-10-23   2019-10-24  \
Attributes Symbols                                                       
open       MSFT          138.45       138.97       136.88       139.39   
           AMZN         1769.66      1788.15      1761.30      1771.09   
high       MSFT          138.50       140.01       137.45       140.42   
           AMZN         1785.88      1789.78      1770.05      1788.34   
low        MSFT          137.01       136.26       135.61       138.67   
           AMZN         1765.00      1762.00      1742.00      1760.27   
close      MSFT          138.43       136.37       137.24       139.94   
           AMZN         1785.66      1765.73      1762.17      1780.78   
volume     MSFT     20716039.00  27935270.00  31380309.00  37278399.00   
           AMZN      2224902.00   2234425.00   2190380.00   5204350.00   

date                 2019-10-25   2019-10-28   2019-10-29   2019-10-30  
Attributes Symbols                                                      
open       MSFT          139.34       144.40       144.08       143.52  
           AMZN         1697.55      1748.06      1774.81      1760.24  
high       MSFT          141.14       145.67       144.50       145.00  
           AMZN    

In [17]:
sp500_df = pd.DataFrame()

In [18]:
sp500_df

""


In [19]:
sp500_df = pd.concat([sp500_df, stock_df])

In [22]:
sp500_df = pd.concat([sp500_df, stock_df])

In [23]:
sp500_df

,open,high,low,close,volume
date,,,,,
2014-10-31,46.94,46.97,46.48,46.95,35849656
2014-11-03,46.89,47.46,46.73,47.44,23130397
2014-11-04,47.30,47.73,47.25,47.57,21530813
2014-11-05,47.80,47.90,47.26,47.86,22449594
2014-11-06,47.86,48.86,47.79,48.70,33037841
...,...,...,...,...,...
2019-10-24,1771.09,1788.34,1760.27,1780.78,5204350
2019-10-25,1697.55,1764.21,1695.00,1761.33,9626402
2019-10-28,1748.06,1778.70,1742.50,1777.08,3708851


In [56]:
### Get data for all SP500 companies
SP500_stock_df = web.DataReader(sp500_wikipedia[:100],'iex', '2019-10-30', '2019-10-31')

In [57]:
SP500_stock_df

Attributes    open                                                      \
Symbols        MMM    ABT   ABBV    ABMD     ACN   ATVI    ADBE    AMD   
date                                                                     
2019-10-30  168.23  83.32  78.51  184.07  185.15  55.25  273.25  32.93   

Attributes                 ...   volume                                     \
Symbols        AAP    AES  ...     CPRI      CAH     KMX      CCL      CAT   
date                       ...                                               
2019-10-30  163.99  17.08  ...  2299671  2807661  829045  4253720  3948056   

Attributes                                            
Symbols       CBOE    CBRE      CBS      CDW      CE  
date                                                  
2019-10-30  575191  846875  5622034  1035225  619110  

[1 rows x 500 columns]

In [41]:
# Define the start and end date
end_data = datetime.now()
start_time = datetime(end_data.year - 5, end_data.month , end_data.day)
print(end_data)
print(start_time)

2019-10-31 14:36:02.931490
2014-10-31 00:00:00


In [53]:
stock_df = web.DataReader('CELG','iex', '2019-10-30', end_data)

In [54]:
stock_df

,open,high,low,close,volume
date,,,,,
2019-10-30,106.47,107.33,106.31,107.17,3516614


In [50]:
sp500_wikipedia = ['MMM',
'ABT',
'ABBV',
'ABMD',
'ACN',
'ATVI',
'ADBE',
'AMD',
'AAP',
'AES',
'AMG',
'AFL',
'A',
'APD',
'AKAM',
'ALK',
'ALB',
'ARE',
'ALXN',
'ALGN',
'ALLE',
'AGN',
'ADS',
'LNT',
'ALL',
'GOOGL',
'GOOG',
'MO',
'AMZN',
'AMCR',
'AEE',
'AAL',
'AEP',
'AXP',
'AIG',
'AMT',
'AWK',
'AMP',
'ABC',
'AME',
'AMGN',
'APH',
'ADI',
'ANSS',
'ANTM',
'AON',
'AOS',
'APA',
'AIV',
'AAPL',
'AMAT',
'APTV',
'ADM',
'ARNC',
'ANET',
'AJG',
'AIZ',
'ATO',
'T',
'ADSK',
'ADP',
'AZO',
'AVB',
'AVY',
'BKR',
'BLL',
'BAC',
'BK',
'BAX',
'BBT',
'BDX',
'BRK.B',
'BBY',
'BIIB',
'BLK',
'HRB',
'BA',
'BKNG',
'BWA',
'BXP',
'BSX',
'BMY',
'AVGO',
'BR',
'BF.B',
'CHRW',
'COG',
'CDNS',
'CPB',
'COF',
'CPRI',
'CAH',
'KMX',
'CCL',
'CAT',
'CBOE',
'CBRE',
'CBS',
'CDW',
'CE',
'CELG',
'CNC',
'CNP',
'CTL',
'CERN',
'CF',
'SCHW',
'CHTR',
'CVX',
'CMG',
'CB',
'CHD',
'CI',
'XEC',
'CINF',
'CTAS',
'CSCO',
'C',
'CFG',
'CTXS',
'CLX',
'CME',
'CMS',
'KO',
'CTSH',
'CL',
'CMCSA',
'CMA',
'CAG',
'CXO',
'COP',
'ED',
'STZ',
'COO',
'CPRT',
'GLW',
'CTVA',
'COST',
'COTY',
'CCI',
'CSX',
'CMI',
'CVS',
'DHI',
'DHR',
'DRI',
'DVA',
'DE',
'DAL',
'XRAY',
'DVN',
'FANG',
'DLR',
'DFS',
'DISCA',
'DISCK',
'DISH',
'DG',
'DLTR',
'D',
'DOV',
'DOW',
'DTE',
'DUK',
'DRE',
'DD',
'DXC',
'ETFC',
'EMN',
'ETN',
'EBAY',
'ECL',
'EIX',
'EW',
'EA',
'EMR',
'ETR',
'EOG',
'EFX',
'EQIX',
'EQR',
'ESS',
'EL',
'EVRG',
'ES',
'RE',
'EXC',
'EXPE',
'EXPD',
'EXR',
'XOM',
'FFIV',
'FB',
'FAST',
'FRT',
'FDX',
'FIS',
'FITB',
'FE',
'FRC',
'FISV',
'FLT',
'FLIR',
'FLS',
'FMC',
'F',
'FTNT',
'FTV',
'FBHS',
'FOXA',
'FOX',
'BEN',
'FCX',
'GPS',
'GRMN',
'IT',
'GD',
'GE',
'GIS',
'GM',
'GPC',
'GILD',
'GL',
'GPN',
'GS',
'GWW',
'HAL',
'HBI',
'HOG',
'HIG',
'HAS',
'HCA',
'HCP',
'HP',
'HSIC',
'HSY',
'HES',
'HPE',
'HLT',
'HFC',
'HOLX',
'HD',
'HON',
'HRL',
'HST',
'HPQ',
'HUM',
'HBAN',
'HII',
'IEX',
'IDXX',
'INFO',
'ITW',
'ILMN',
'IR',
'INTC',
'ICE',
'IBM',
'INCY',
'IP',
'IPG',
'IFF',
'INTU',
'ISRG',
'IVZ',
'IPGP',
'IQV',
'IRM',
'JKHY',
'JEC',
'JBHT',
'SJM',
'JNJ',
'JCI',
'JPM',
'JNPR',
'KSU',
'K',
'KEY',
'KEYS',
'KMB',
'KIM',
'KMI',
'KLAC',
'KSS',
'KHC',
'KR',
'LB',
'LHX',
'LH',
'LRCX',
'LW',
'LVS',
'LEG',
'LDOS',
'LEN',
'LLY',
'LNC',
'LIN',
'LKQ',
'LMT',
'L',
'LOW',
'LYB',
'MTB',
'MAC',
'M',
'MRO',
'MPC',
'MKTX',
'MAR',
'MMC',
'MLM',
'MAS',
'MA',
'MKC',
'MXIM',
'MCD',
'MCK',
'MDT',
'MRK',
'MET',
'MTD',
'MGM',
'MCHP',
'MU',
'MSFT',
'MAA',
'MHK',
'TAP',
'MDLZ',
'MNST',
'MCO',
'MS',
'MOS',
'MSI',
'MSCI',
'MYL',
'NDAQ',
'NOV',
'NTAP',
'NFLX',
'NWL',
'NEM',
'NWSA',
'NWS',
'NEE',
'NLSN',
'NKE',
'NI',
'NBL',
'JWN',
'NSC',
'NTRS',
'NOC',
'NCLH',
'NRG',
'NUE',
'NVDA',
'NVR',
'ORLY',
'OXY',
'OMC',
'OKE',
'ORCL',
'PCAR',
'PKG',
'PH',
'PAYX',
'PYPL',
'PNR',
'PBCT',
'PEP',
'PKI',
'PRGO',
'PFE',
'PM',
'PSX',
'PNW',
'PXD',
'PNC',
'PPG',
'PPL',
'PFG',
'PG',
'PGR',
'PLD',
'PRU',
'PEG',
'PSA',
'PHM',
'PVH',
'QRVO',
'PWR',
'QCOM',
'DGX',
'RL',
'RJF',
'RTN',
'O',
'REG',
'REGN',
'RF',
'RSG',
'RMD',
'RHI',
'ROK',
'ROL',
'ROP',
'ROST',
'RCL',
'CRM',
'SBAC',
'SLB',
'STX',
'SEE',
'SRE',
'SHW',
'SPG',
'SWKS',
'SLG',
'SNA',
'SO',
'LUV',
'SPGI',
'SWK',
'SBUX',
'STT',
'SYK',
'STI',
'SIVB',
'SYMC',
'SYF',
'SNPS',
'SYY',
'TMUS',
'TROW',
'TTWO',
'TPR',
'TGT',
'TEL',
'FTI',
'TFX',
'TXN',
'TXT',
'TMO',
'TIF',
'TWTR',
'TJX',
'TSCO',
'TDG',
'TRV',
'TRIP',
'TSN',
'UDR',
'ULTA',
'USB',
'UAA',
'UA',
'UNP',
'UAL',
'UNH',
'UPS',
'URI',
'UTX',
'UHS',
'UNM',
'VFC',
'VLO',
'VAR',
'VTR',
'VRSN',
'VRSK',
'VZ',
'VRTX',
'VIAB',
'V',
'VNO',
'VMC',
'WAB',
'WMT',
'WBA',
'DIS',
'WM',
'WAT',
'WEC',
'WCG',
'WFC',
'WELL',
'WDC',
'WU',
'WRK',
'WY',
'WHR',
'WMB',
'WLTW',
'WYNN',
'XEL',
'XRX',
'XLNX',
'XYL',
'YUM',
'ZBH',
'ZION',
'ZTS']